In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import sys,os,time,datetime,warnings,math,itertools

import torch
import pandas as pd

import tqdm

In [3]:
sys.path.append('../../7_HELPERFUNCTIONS/')
from WorldSimulators.TransferChannels import MIMOTimeInvariant

In [199]:
channel = MIMOTimeInvariant(5,10)

num_RX = channel.H.shape[0]
num_TX = channel.H.shape[1]

R = np.zeros([num_RX,num_TX])

for k in range(num_TX):
    v = np.zeros(num_TX)
    v[k] = 1

    R[:,k] = channel.transfer_v(v).transpose().reshape(num_RX)

invec = rnd.randn(num_RX)/(num_RX*num_TX)
outvec = channel.transfer_v((np.linalg.inv(R.transpose()@R)@R.transpose())@invec)
c_err = np.linalg.norm(invec-outvec,2)/num_RX

print(c_err)

0.024366657593369607


In [242]:
TESTSET = rnd.randn(num_RX,20)/(num_RX*num_TX)

RESULTS = []
for k in range(TESTSET.shape[1]):
    RESULTS.append(channel.transfer_v((np.linalg.inv(R.transpose()@R)@R.transpose())@TESTSET[:,k]).reshape(-1))
RESULTS = np.array(RESULTS).transpose()

In [243]:
C = np.zeros([TESTSET.shape[1],TESTSET.shape[1]])
for m in range(TESTSET.shape[1]):
    for n in range(TESTSET.shape[1]):
        C[m,n] = TESTSET[:,m]@RESULTS[:,n]

In [244]:
C

array([[ 7.83809740e-03, -1.80376924e-03,  5.42513633e-04,
        -2.95277686e-03,  5.13430567e-03,  5.70404500e-03,
        -1.08378919e-03, -4.55706020e-03, -5.21762951e-03,
        -4.86080254e-03,  2.92982857e-03,  2.44397778e-03,
         3.08910447e-03,  2.03903348e-03, -6.46741071e-03,
        -3.30966005e-03, -3.73925375e-03,  3.89771042e-03,
         5.54267907e-03,  8.84392960e-04],
       [-7.13786511e-03,  5.89606805e-03,  9.28812482e-04,
         1.34062456e-03, -9.79021577e-04, -7.46706276e-03,
         3.71415745e-03,  5.38217616e-03,  2.48966701e-03,
         5.58578186e-03, -1.87415539e-03, -1.88891834e-03,
        -5.47535084e-03, -4.62876408e-03,  6.19803038e-03,
         3.71642367e-03,  3.91088964e-03,  9.06631652e-05,
        -7.95803295e-03, -9.78989230e-04],
       [-8.31200753e-06, -1.24198955e-03,  1.42202617e-03,
         1.23355498e-04, -5.38355874e-04, -8.21469566e-04,
         8.94145229e-04,  1.41176989e-03,  1.23148028e-03,
         1.30648988e-03, -1.2

In [410]:
class Precoder(torch.nn.Module):
    def __init__(self,channel):
        super(Precoder,self).__init__()
        self.l0 = torch.nn.Linear(channel.out_size,16)
        self.l1 = torch.nn.Linear(16,channel.in_size)
    def forward(self,x):
        x = torch.nn.functional.relu(self.l0(x))
        x = torch.nn.functional.tanh(self.l1(x))
        return x

In [429]:
precoder = Precoder(channel) 

#optimizer = torch.optim.Adam(precoder.parameters(),lr=0.01)
#criterion = torch.nn.MSELoss()

num_epochs = 5
val_size = 0.2

batch_size = 1

epsilon = 0.1

In [430]:
R = []

y_target = torch.tensor(rnd.randn(n)).float()

for k in range(1000):

    if (k+1)%100==0: print('Run {}'.format(k+1))
    
    
    x_hat = precoder(y_target)
    
    if torch.isnan(x_hat).any(): break
    
    y_measured = channel.transfer_v(np.array(x_hat.tolist()))

    penalty = np.linalg.norm(np.array(y_target)-y_measured,axis=1)
    
    c_grad = torch.autograd.grad(x_hat.sum().log(),precoder.parameters())
    for layer,p in enumerate(precoder.parameters()):
        p.data += c_grad[layer]*epsilon*penalty[0]

    R.append(penalty)

Run 100
Run 200
Run 300
Run 400
Run 500
Run 600
Run 700
Run 800
Run 900
Run 1000


In [431]:
R

[array([4.09842089]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.

In [432]:
y_target

tensor([-2.1669,  0.7192, -1.1355,  1.0028, -0.5816])

In [433]:
y_measured

array([[1.89430367, 1.1793932 , 1.52419856, 0.99367888, 2.21066926]])

In [422]:
np.linalg.norm((np.array(y_target)-y_measured)[0])

6.459812584828123

In [424]:
penalty[0]

6.459812584828123